In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import kerastuner as kt
import pickle
import joblib
from scipy import stats
from numpy import load
from numpy import save
from tensorflow.keras import regularizers
import seaborn as sns
import scipy

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
sns.set_style("white")

### set seed

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

### Load data

In [ ]:
train_norm = pd.read_pickle("../final_data/train_norm.pkl")
valid_norm = pd.read_pickle("../final_data/valid_norm.pkl")
test_norm = pd.read_pickle("../final_data/test_norm.pkl")
all_train_norm = pd.read_pickle("../final_data/all_train_norm.pkl")

scaler_x = joblib.load("../final_data/scaler_x.save") 
scaler_y = joblib.load("../final_data/scaler_y.save") 

scaler_train_x = joblib.load("../final_data/scaler_train_x.save") 
scaler_train_y = joblib.load("../final_data/scaler_train_y.save")

train_data = pd.read_pickle("../final_data/train_unsc.pkl")
valid_data = pd.read_pickle("../final_data/valid_unsc.pkl")
test_data = pd.read_pickle("../final_data/test_unsc.pkl")
all_train_data = pd.read_pickle("../final_data/all_train_unsc.pkl")

with open ('../final_data/label_keys_ts', 'rb') as fp:
    label_keys = pickle.load(fp)
with open ('../final_data/feature_keys_ts', 'rb') as fp:
    feature_keys = pickle.load(fp)    

In [ ]:
all_train_x = all_train_norm[feature_keys]
all_train_y = all_train_norm[label_keys]
test_x = test_norm[feature_keys]
test_y = test_norm[label_keys]
valid_x = valid_norm[feature_keys]
valid_y = valid_norm[label_keys]
train_x = train_norm[feature_keys]
train_y = train_norm[label_keys]

### Baseline

In [ ]:
train_x_unsc = all_train_data[feature_keys]
train_y_unsc = all_train_data[label_keys]
test_x_unsc = test_data[feature_keys]
test_y_unsc = test_data[label_keys]

In [ ]:
train_pred_base = test_y_unsc.copy()
train_pred_base['TempMittelLsInC'] = train_pred_base['TempMittelLsInC'].mean(axis=0)
train_pred_base['TempMittelFsInC'] = train_pred_base['TempMittelFsInC'].mean(axis=0)
train_pred_base = train_pred_base.to_numpy()

#### RMSE

In [ ]:
mse = mean_squared_error(test_y_unsc.to_numpy(), train_pred_base)                             
np.sqrt(mse)

## Loop function

In [ ]:
def test_10_times(model, epochs, batch_size):
    runs = 10
    test_score = list()
    train_score = list()
 
    for i in range(runs):

        np.random.seed(123 + i)
        tf.random.set_seed(123 + i)



        history = model.fit(all_train_x, all_train_y,
                        epochs=epochs,
                        batch_size=batch_size,
                       )

        y_pred_test = model.predict(test_x)
        y_test_unsc = scaler_y.inverse_transform(test_y)
        y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

        y_pred_train = model.predict(all_train_x)
        y_train_unsc = scaler_y.inverse_transform(all_train_y)
        y_pred_train_unsc = scaler_y.inverse_transform(y_pred_train)
        
        mse_test = np.sqrt(mean_squared_error(y_pred_test_unsc, y_test_unsc))  
        test_score.append(mse_test)

        mse_train = np.sqrt(mean_squared_error(y_pred_train_unsc, y_train_unsc))  
        train_score.append(mse_train)

    return  train_score, test_score


### Different Models

## simple model

In [ ]:
def build_model_simple_mlp():
    model = keras.models.Sequential([
        keras.layers.Dense(32, activation='relu',kernel_initializer='he_normal', input_shape=train_x.shape[1:]),
        keras.layers.Dense(2)
    ])
    model.compile(loss='mse',
                  optimizer=keras.optimizers.Adam(lr=1e-3),
                 )
    return model

### Training

In [ ]:
simple_model = build_model_simple_mlp()
history_simple_model = simple_model.fit(train_x, train_y,
                    epochs=100,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                   )

In [ ]:
simple_mlp_ts = keras.models.load_model("/home/di40438/bachelorarbeit/final_data/simple_mlp_ts_train")
hist_simple_mlp_ts = pd.read_pickle("/home/di40438/bachelorarbeit/final_data/hist_simple_mlp_ts_train.pkl")

In [ ]:
pd.DataFrame(hist_simple_mlp_ts).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

### Performance after applying early stopping

In [ ]:
## Calculate RMSE on training set
y_pred_train = simple_mlp_ts.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)

np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
## Calculate RMSE on validation set
y_pred_val = simple_mlp_ts.predict(valid_x)
y_val_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_val_unsc = scaler_train_y.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

### Testing

In [ ]:
simple_model = build_model_simple_mlp()

history_simple_model = simple_model.fit(all_train_x, all_train_y,
                    epochs=87,
                    batch_size=32,
                   )

In [ ]:
simple_model_test = keras.models.load_model("/home/di40438/bachelorarbeit/final_data/models/simple_mlp_ts_test")

In [ ]:
## Calculate RMSE on test set
y_pred_test = simple_model_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc))                       

### Test 10 times

In [ ]:
simple_model = build_model_simple_mlp()
epochs = 87
batch_size = 32
train_score, test_score = test_10_times(simple_model, epochs, batch_size)

In [ ]:
rmse_train_simple_mlp_ts = pd.read_pickle("../final_data/rmse_train_simple_mlp_ts.pkl")
rmse_test_simple_mlp_ts = pd.read_pickle("../final_data/rmse_test_mlp_ts_simple_model.pkl")

In [ ]:
rmse_train_simple_mlp_ts.describe()

In [ ]:
rmse_test_simple_mlp_ts.describe()

## Simple Model with Dropout

In [ ]:
def build_model_simple_dropout():
    model_reg = keras.models.Sequential([
        keras.layers.Dense(32, activation='relu',kernel_initializer='he_normal', input_shape=train_x.shape[1:]),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])
    model_reg.compile(loss='mse',
                 
        optimizer=keras.optimizers.Adam(lr=1e-3),
                  
                 )
    return model_reg

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
model_simple_dropout = build_model_simple_dropout()

history_simple_dropout = model_simple_dropout.fit(train_x, train_y,
                    epochs=100,                         
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

Load model

In [ ]:
hist_simple_dropout = pd.read_pickle("../final_data/hist_simple_mlp_ts_drop.pkl")
model_simple_dropout = keras.models.load_model("../final_data/models/simple_mlp_ts_drop")

In [ ]:
pd.DataFrame(hist_simple_dropout).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 0.005)

In [ ]:
y_pred_train = model_simple_dropout.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)

np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_val = model_simple_dropout.predict(valid_x)
y_val_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_val_unsc = scaler_train_y.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

## Simple Model with weight regularization

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

def build_model_simple_weight_reg():
    model_kernel = keras.models.Sequential([
        keras.layers.Dense(32, activation='relu',kernel_initializer='he_normal', input_shape=train_x.shape[1:],
                          kernel_regularizer=regularizers.l2(1e-4)),
        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])
    model_kernel.compile(loss='mse',
                  optimizer=keras.optimizers.Adam(lr=1e-3),
                 
                 )
    return model_kernel

### Training

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
model_simple_weight_reg = build_model_simple_weight_reg()
history_simple_weight_reg = model_simple_weight_reg.fit(train_x, train_y,
                    epochs=200,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

Load model

In [ ]:
hist_simple_weight_reg = pd.read_pickle("/home/di40438/bachelorarbeit/final_data/hist_simple_mlp_ts_weight.pkl")
model_simple_weight_reg = keras.models.load_model("/home/di40438/bachelorarbeit/final_data/models/simple_mlp_ts_weight")

In [ ]:
pd.DataFrame(hist_simple_weight_reg).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 0.005)

In [ ]:
y_pred_train = model_simple_weight_reg.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)

np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_val = model_simple_weight_reg.predict(valid_x)
y_val_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_val_unsc = scaler_train_y.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

## Model with Dropout

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

def build_model_dropout_tuned():
    model_reg = keras.models.Sequential([
        keras.layers.Dense(32, activation='relu',kernel_initializer='he_normal',
                           input_shape=train_x.shape[1:],
                           kernel_constraint=keras.constraints.max_norm(max_value=1),),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(32, activation='relu',kernel_initializer='he_normal',
                          kernel_constraint=keras.constraints.max_norm(max_value=1),),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(32, activation='relu',kernel_initializer='he_normal',
                           kernel_constraint=keras.constraints.max_norm(max_value=1),),
        keras.layers.Dropout(0.2),
        
        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])
    model_reg.compile(loss='mse',       
        optimizer=keras.optimizers.Adam(lr=1e-3),         
                 )
    return model_reg

### Training

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

model_dropout = build_model_dropout_tuned()
history_dropout_model = model_dropout.fit(train_x, train_y,
                    epochs=300,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback],
                                         )

Load model

In [ ]:
hist_model_dropout_train = pd.read_pickle("../final_data/hist_mlp_ts_dropout.pkl")
model_dropout_train = keras.models.load_model("../final_data/models/mlp_ts_dropout")

In [ ]:
pd.DataFrame(hist_model_dropout_train).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 0.005)

In [ ]:
y_pred_train = model_dropout_train.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)

np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_val = model_dropout_train.predict(valid_x)
y_val_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_val_unsc = scaler_train_y.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

### Testing

In [ ]:
model_dropout_test = build_model_dropout_tuned()

history_model_dropout_test = model_dropout_test.fit(all_train_x, all_train_y,
                    epochs=206,
                    batch_size=32,
                   )

Load model

In [ ]:
model_dropout_test = keras.models.load_model("../final_data/models/mlp_ts_dropout_test")

In [ ]:
y_pred_test = model_dropout_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc))                       

In [ ]:
model_dropout = build_model_dropout_tuned()
epochs = 206
batch_size = 32
train_score, test_score = test_10_times(model_dropout, epochs, batch_size)

In [ ]:
rmse_train_model_dropout = pd.read_pickle("../final_data/rmse_train_model_dropout.pkl")
rmse_test_model_dropout = pd.read_pickle("../final_data/rmse_test_model_dropout.pkl")

In [ ]:
rmse_train_model_dropout.describe()

In [ ]:
rmse_test_model_dropout.describe()

### MLP Dropout Hyperband

### Keras Tuner implementation of Hyperband
Hyperparameter search may take a while even with limited data, resulting architectur when used with all data is provided next

In [ ]:
## MLP with Dropout
def model_builder_mlp (hp):
    
    model = keras.models.Sequential()
   
    hp_max_norm = hp.Choice("max_norm",values = [0, 1, 2])
    hp_units = hp.Choice("unit",values = [16, 32, 64, 128, 192 ]) 
    hp_dropout = hp.Choice("mlp_layer_dropout",values = [ 0.2, 0.3])
    hp_number_of_layers = hp.Int("layers",min_value=0,max_value=3,step=1)
    
    model.add(keras.layers.Dense(
                            units= hp_units,
                            kernel_constraint=keras.constraints.max_norm(max_value=hp_max_norm),                    
                            activation='relu',
                            input_shape=train_x.shape[1:]     
                     ))
    
    model.add(keras.layers.Dropout(hp_dropout))
    
    for i in range(hp_number_of_layers):
        model.add(keras.layers.Dense(
                        units = hp_units,
                        kernel_constraint=keras.constraints.max_norm(max_value=hp_max_norm),
                        activation='relu',
                        kernel_initializer='he_normal'
                        ))
        
        model.add(keras.layers.Dropout(hp_dropout))
    
    model.add(keras.layers.Dense(2, kernel_initializer='he_normal'))
        
    mlp_learning_rate = hp.Choice('learning_rate', values = [ 1e-3, 1e-4])
    optimizer = keras.optimizers.Adam(lr=mlp_learning_rate)
    
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

class MyTuner(kt.tuners.Hyperband):
  def run_trial(self, trial, *args, **kwargs):
    kwargs['batch_size'] = trial.hyperparameters.Choice("batch_size",values = [32, 128, 512])
    super(MyTuner, self).run_trial(trial, *args, **kwargs)

In [ ]:
# tuner = MyTuner(model_builder_mlp,
#                 objective='val_loss',
#                 max_epochs = 100,
#                 factor = 3,
#                 seed = 42,
#                 directory = "k",
#                 project_name = 'k1')


# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

# tuner.search(train_x,
#              train_y,
#              epochs = 150,
#              validation_data = (valid_x, valid_y),
#              callbacks = [stop_early]
#             )

### Training

#### Either: Keras Tuner implementation of Hyperband

In [ ]:
tuner_hyper_dropout = pickle.load(open("/home/di40438/bachelorarbeit/final_data/models/mlp_ts_drop_hyper.pkl",'rb'))

print(tuner_hyper_dropout.get_best_hyperparameters()[0].values)

In [ ]:
best_hps = tuner_hyper_dropout.get_best_hyperparameters(num_trials=1)[0]
hypermodel_dropout = tuner_hyper_dropout.hypermodel.build(best_hps)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

# Retrain the model
history_hypermodel_dropout = hypermodel_dropout.fit(train_x, train_y,
                    epochs=200,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[early_stop]
                   )

#### Or: Rebuild best Hyperband model manually

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

def build_hypermodel_dropout():
    model_reg = keras.models.Sequential([
        keras.layers.Dense(128, activation='relu',kernel_initializer='he_normal',
                           input_shape=train_x.shape[1:],
                           kernel_constraint=keras.constraints.max_norm(max_value=1),),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(128, activation='relu',kernel_initializer='he_normal',
                          kernel_constraint=keras.constraints.max_norm(max_value=1),),
        keras.layers.Dropout(0.2),

        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])
    model_reg.compile(loss='mse',
        optimizer=keras.optimizers.Adam(lr=1e-3),     
                 )
    return model_reg

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

hypermodel_dropout = build_hypermodel_dropout()

history_hypermodel_dropout = hypermodel_dropout.fit(train_x, train_y,
                    epochs=200,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

Load model

In [ ]:
hist_hypermodel_dropout_train = pd.read_pickle("../final_data/hist_hyper_mlp_ts_drop_train.pkl")
hypermodel_dropout_train = keras.models.load_model("../final_data/models/hyper_mlp_ts_drop_train")

In [ ]:
pd.DataFrame(hist_hypermodel_dropout_train).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 0.0025)

In [ ]:
y_pred_train = hypermodel_dropout_train.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_val = hypermodel_dropout_train.predict(valid_x)
y_val_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_val_unsc = scaler_train_y.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

### Testing
#### Either: rebuild Keras Tuner model

In [ ]:
best_hps = tuner_hyper_dropout.get_best_hyperparameters(num_trials=1)[0]
hypermodel_dropout_test = tuner_hyper_dropout.hypermodel.build(best_hps)

# Retrain the model
history_hypermodel_dropout = hypermodel_dropout_test.fit(all_train_x, all_train_y,
                    epochs=92,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

#### Or: rebuild best model manually

In [ ]:
hypermodel_dropout = build_hypermodel_dropout()

history_hypermodel_dropout = hypermodel_dropout.fit(all_train_x, all_train_y,
                    epochs=92,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

In [ ]:
hypermodel_dropout_test = keras.models.load_model("../final_data/models/hyper_mlp_ts_drop_test")

In [ ]:
y_pred_test = hypermodel_dropout_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc))                       

### Test 10 times

In [ ]:
hypermodel_dropout = build_hypermodel_dropout()
epochs = 92
batch_size = 32
train_score, test_score = test_10_times(hypermodel_dropout, epochs, batch_size)

In [ ]:
rmse_train_mlp_ts_hyperband_dropout = pd.read_pickle("../final_data/rmse_train_mlp_ts_hyperband_dropout.pkl")
rmse_test_mlp_ts_hyperband_dropout = pd.read_pickle("../final_data/rmse_test_mlp_ts_hyperband_dropout.pkl")

In [ ]:
rmse_train_mlp_ts_hyperband_dropout.describe()

In [ ]:
rmse_test_mlp_ts_hyperband_dropout.describe()

## Model with weight regularization

In [ ]:
def build_model_weight_reg_tuned():
    model_kernel = keras.models.Sequential([
        keras.layers.Dense(48, activation='relu',kernel_initializer='he_normal', input_shape=train_x.shape[1:],
                          kernel_regularizer=regularizers.l2(1e-4),
                          
                          ),
        keras.layers.Dense(48, activation='relu',kernel_initializer='he_normal', 
                          kernel_regularizer=regularizers.l2(1e-4),
                         
                          ),
        keras.layers.Dense(48, activation='relu',kernel_initializer='he_normal', 
                          kernel_regularizer=regularizers.l2(1e-4),
                       
                          ),
        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])
    model_kernel.compile(loss='mse',
                  optimizer=keras.optimizers.Adam(lr=1e-3),
                 
                 )
    return model_kernel

### Training

In [ ]:
model_weight_reg_train = build_model_weight_reg_tuned()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history_model_weight_reg_train = model_weight_reg_train.fit(train_x, train_y,
                    epochs=300,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                                                           )

Load model

In [ ]:
hist_weigh_reg_train = pd.read_pickle("../final_data/hist_weigh_reg_train.pkl")
model_weight_reg_train = keras.models.load_model("../final_data/models/mlp_ts_weight_reg_train")

In [ ]:
pd.DataFrame(hist_weigh_reg_train).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 0.005)

In [ ]:
y_pred_train = model_weight_reg_train.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_val = model_weight_reg_train.predict(valid_x)
y_val_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_val_unsc = scaler_train_y.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

### Testing

In [ ]:
model_weight_reg_test = build_model_weight_reg_tuned()

model_weight_reg_test.fit(all_train_x, all_train_y,
                    epochs=147,
                    batch_size=32,
                         )

In [ ]:
model_weight_reg_test = keras.models.load_model("../final_data/models/mlp_ts_weight_reg_test")

In [ ]:
y_pred_train = model_weight_reg_test.predict(all_train_x)
y_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_train_unsc = scaler_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_test = model_weight_reg_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc))                       

### Test 10 times

In [ ]:
model_weight_reg = build_model_weight_reg_tuned()
epochs = 147
batch_size = 32
train_score, test_score = test_10_times(model_weight_reg, epochs, batch_size)

In [ ]:
rmse_train_mlp_ts_weight_reg = pd.read_pickle("../final_data/rmse_train_mlp_ts_weight_reg.pkl")
rmse_test_mlp_ts_weight_reg = pd.read_pickle("../final_data/rmse_test_mlp_ts_weight_reg.pkl")

In [ ]:
rmse_train_mlp_ts_weight_reg.describe()

In [ ]:
rmse_test_mlp_ts_weight_reg.describe()

### MLP Weight Regularization Hyperband

### Keras Tuner implementation of Hyperband

In [ ]:
def model_builder_mlp (hp):
    
    model = keras.models.Sequential()
    hp_kernel_reg = hp.Choice("mlp_layer_kernel_reg",values = [1e-3, 1e-4])
    hp_units = hp.Choice("unit",values = [16, 32, 64, 128, 192 ]) 
    hp_number_of_layers = hp.Choice("layers",values = [0,1,2,3]) 
    
    model.add(keras.layers.Dense(
                            units= hp_units, 
                            kernel_regularizer=regularizers.l2(hp_kernel_reg),          
                            activation='relu',
                            input_shape=train_x.shape[1:]     
                     ))

    for i in range(hp_number_of_layers):
        model.add(keras.layers.Dense(
                        units = hp_units,
                        kernel_regularizer=regularizers.l2(hp_kernel_reg),
                        activation='relu',
                        kernel_initializer='he_normal'
                        ))

    model.add(keras.layers.Dense(2, kernel_initializer='he_normal'))
        
    mlp_learning_rate = hp.Choice('learning_rate', values = [ 1e-3, 1e-4])
    optimizer = keras.optimizers.Adam(lr=mlp_learning_rate)
    
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
              
              
class MyTuner(kt.tuners.Hyperband):
  def run_trial(self, trial, *args, **kwargs):
    kwargs['batch_size'] = trial.hyperparameters.Choice("batch_size",values = [32, 128, 512])
    super(MyTuner, self).run_trial(trial, *args, **kwargs)

Only needs to be executed if Hyperband should search again, results can be loaded

In [ ]:
# tuner_hyper_weight = MyTuner(model_builder_mlp,
#                    # optimizer=keras.optimizers.Adam(1e-3),
#                     #loss='mean_squared_error',
#                     objective='val_loss',
#                     max_epochs = 100,
#                     factor = 3,
#                     seed = 42,
#                     directory = "../data/models",
#                     project_name = 'hyperband_mlp_ts_weight')

# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

# tuner_hyper_weight.search(train_x,
#              train_y,
#              epochs = 150,
#              validation_data = (valid_x, valid_y),
#              callbacks = [stop_early]
#             )

### Training

#### Either: Keras Tuner implementation of Hyperband

Load Tuner if Hyperband was not run, if Hyperband was run, do not execute

In [ ]:
tuner_hyper_weight = pickle.load(open("../final_data/models/hyper_mlp_ts_weight.pkl",'rb'))

If Hyperband was run execute here

In [ ]:
print(tuner_hyper_weight.get_best_hyperparameters()[0].values)

In [ ]:
best_hps = tuner_hyper_weight.get_best_hyperparameters(num_trials=1)[0]
hypermodel_weight_reg = tuner_hyper_weight.hypermodel.build(best_hps)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

# Retrain the model
history_hypermodel_weight_reg = hypermodel_weight_reg.fit(train_x, train_y,
                    epochs=200,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[early_stop]
                   )

#### Or: Rebuild best Hyperband model manually

In [ ]:
def build_hypermodel_weight_reg():
    model_reg = keras.models.Sequential([
        keras.layers.Dense(320, activation='relu',kernel_initializer='he_normal',
                           input_shape=train_x.shape[1:],
                            kernel_regularizer=regularizers.l2(0.0001)),
        keras.layers.Dense(320, activation='relu',kernel_initializer='he_normal',
                           kernel_regularizer=regularizers.l2(0.0001)),
        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])
    model_reg.compile(loss='mse',
        optimizer=keras.optimizers.Adam(lr=1e-3),     
                 )
    return model_reg

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

hypermodel_weight_reg = build_hypermodel_weight_reg()

history_hypermodel_weight_reg = hypermodel_weight_reg.fit(train_x, train_y,
                    epochs=200,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

Load model

In [ ]:
hist_hypermodel_weigh_reg_train = pd.read_pickle("../final_data/hist_hypermodel_mlp_ts_weight.pkl")
hypermodel_weight_reg_train = keras.models.load_model("../final_data/models/hypermodel_mlp_ts_weight_train")

In [ ]:
pd.DataFrame(hist_hypermodel_weigh_reg_train).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 0.003)

In [ ]:
y_pred_train = hypermodel_weight_reg_train.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_val = hypermodel_weight_reg_train.predict(valid_x)
y_val_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_val_unsc = scaler_train_y.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

### Testing
#### Either: rebuild Keras Tuner model

In [ ]:
best_hps = tuner_hyper_weight.get_best_hyperparameters(num_trials=1)[0]
hypermodel_weight_reg = tuner_hyper_weight.hypermodel.build(best_hps)

# Retrain the model
history_hypermodel_weight_reg = hypermodel_weight_reg.fit(all_train_x, all_train_y,
                    epochs=109,
                    batch_size=32,
                   )

#### Or: rebuild best model manually

In [ ]:
hypermodel_weight_reg = build_hypermodel_weight_reg()

history_hypermodel_weight_reg = hypermodel_weight_reg.fit(all_train_x, all_train_y,
                    epochs=109,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

Load model

In [ ]:
model_weight_reg_test = keras.models.load_model("../final_data/models/hyper_mlp_ts_weight_test")

In [ ]:
y_pred_test = model_weight_reg_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc))                       

### Test 10 times

In [ ]:
hypermodel_weight_reg = build_hypermodel_weight_reg()
epochs = 109
batch_size = 32
train_score, test_score = test_10_times(hypermodel_dropout, epochs, batch_size)

In [ ]:
rmse_train_mlp_ts_hyperband_weight = pd.read_pickle("/home/di40438/bachelorarbeit/final_data/rmse_train_mlp_ts_hyperband_weight.pkl")
rmse_test_mlp_ts_hyperband_weight = pd.read_pickle("/home/di40438/bachelorarbeit/final_data/rmse_test_mlp_ts_hyperband_weight.pkl")

In [ ]:
rmse_train_mlp_ts_hyperband_weight.describe()

In [ ]:
rmse_test_mlp_ts_hyperband_weight.describe()

## Plot Simple Models

#### If not all models were run: Read saved files

In [ ]:
hist_simple_mlp_ts = pd.read_pickle("../final_data/hist_simple_mlp_ts_train.pkl")
hist_simple_dropout = pd.read_pickle("../final_data/hist_simple_mlp_ts_drop.pkl")
hist_simple_weight_reg = pd.read_pickle("../final_data/hist_simple_mlp_ts_weight.pkl")

In [ ]:
# hist_simple_model = pd.read_pickle("../data/lstm_mlp/mlp_models/hist_simple_model.pkl")
# hist_simple_dropout = pd.read_pickle("../data/lstm_mlp/mlp_models/hist_simple_dropout.pkl")
# hist_model_simple_weight_reg = pd.read_pickle("../data/lstm_mlp/mlp_models/hist_model_simple_weight_reg.pkl")

In [ ]:
fig_simple_mlps, ax = plt.subplots(figsize=(12, 5))
ax.plot(hist_simple_mlp_ts['loss'], label='SimpleMLP_Loss', color='dimgrey' )
ax.plot(hist_simple_mlp_ts['val_loss'], label='SimpleMLP_Valid_Loss', color='black' )
ax.plot(hist_simple_dropout['loss'], label='SimpleDropout_Loss', color='indianred')
ax.plot(hist_simple_dropout['val_loss'], label='SimpleDropout_Valid_Loss', color='red')
ax.plot(hist_simple_weight_reg['loss'], label='SimpleWeightReg_Loss', color='c')
ax.plot(hist_simple_weight_reg['val_loss'], label='SimpleWeightReg_Valid_Loss', color='blue')
ax.set_ylim(0.001, 0.005)
ax.set(xlabel=' Epochs', ylabel='Score')
ax.set_xlabel('Epochs', fontsize=15)
ax.set_ylabel('Score', fontsize=15)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
ax.legend(loc='upper center',prop={'size': 12} )
ax.grid()

### Plotting tuned models

#### If not all models were run: Read saved files

In [ ]:
hist_model_dropout_train = pd.read_pickle("../final_data/hist_mlp_ts_dropout.pkl")
hist_hypermodel_dropout_train = pd.read_pickle("../final_data/hist_hyper_mlp_ts_drop_train.pkl")
hist_weigh_reg_train = pd.read_pickle("../final_data/hist_weigh_reg_train.pkl")
hist_hypermodel_weigh_reg_train = pd.read_pickle("../final_data/hist_hypermodel_mlp_ts_weight.pkl")

In [ ]:
fig_final_mlps, ax = plt.subplots(figsize=(6, 5))
ax.plot(hist_model_dropout_train['loss'], label='Dropout_Training', color='indianred' )
ax.plot(hist_model_dropout_train['val_loss'], label='Dropout_Validation', color='red' )
ax.plot(hist_hypermodel_dropout_train['loss'], label='HyperbandDropout_Training', color='dimgrey')
ax.plot(hist_hypermodel_dropout_train['val_loss'], label='HyperbandDropout_Validation', color='black')
ax.set_ylim(0.001, 0.003)
ax.set(xlabel=' Epochs', ylabel='Score')
ax.set_xlabel('Epochs', fontsize=15)
ax.set_ylabel('Score', fontsize=15)
plt.yticks(np.arange(0.001, 0.003+0.0005, 0.0005), fontsize=13)
plt.xticks(fontsize=13)
ax.legend(loc='upper right',prop={'size': 13} )
ax.grid()

In [ ]:
fig_final_mlps, ax = plt.subplots(figsize=(6, 5))
ax.plot(hist_weigh_reg_train['loss'][:300], label='WeightReg_Training', color='c' )
ax.plot(hist_weigh_reg_train['val_loss'][:300], label='WeightReg_Validation', color='blue' )
ax.plot(hist_hypermodel_weigh_reg_train['loss'][:300], label='HyperbandWeightReg_Training', color='mediumseagreen')
ax.plot(hist_hypermodel_weigh_reg_train['val_loss'][:300], label='HyperbandWeightReg_Validation', color='green')
ax.set_ylim(0.001, 0.003)
ax.set(xlabel=' Epochs', ylabel='Score')
ax.set_xlabel('Epochs', fontsize=15)
ax.set_ylabel('Score', fontsize=15)
plt.yticks(np.arange(0.001, 0.003+0.0005, 0.0005), fontsize=13)
plt.xticks(fontsize=13)
ax.legend(loc='upper right',prop={'size': 13} )
ax.grid()

### Graphical Evaluation

In [ ]:
model_weight_reg_train = keras.models.load_model("../final_data/models/mlp_ts_weight_reg_train")
model_weight_reg_test = keras.models.load_model("../final_data/models/mlp_ts_weight_reg_test")

#### use modell from training for validation loss

In [ ]:
y_pred_valid = model_weight_reg_train.predict(valid_x)
y_valid_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_train_y.inverse_transform(y_pred_valid)
np.sqrt(mean_squared_error(y_valid_unsc, y_pred_valid_unsc))                       

#### use model from testing for all_train and test loss

In [ ]:
y_pred_train = model_weight_reg_test.predict(all_train_x)
y_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_train_unsc = scaler_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_test_final = model_weight_reg_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc_final = scaler_y.inverse_transform(y_pred_test_final)
np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc_final))   

In [ ]:
ig, ax1 = plt.subplots(1, figsize=(5, 5),  sharex=True, sharey=True)
plt.xlim(550, 900)
plt.ylim(550, 900)

ax1.scatter( y_pred_train_unsc, y_train_unsc,  c= 'dodgerblue',s=1)
ax1.plot([550,900],[550,900], c='black')
ax1.grid()
ax1.set_xlabel('Predicted Temperatures [°C] ', fontsize=15)
ax1.set_ylabel('Measured Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.set(xticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(5, 5),  sharex=True, sharey=True)
plt.xlim(550, 900)
plt.ylim(550, 900)

ax1.scatter( y_pred_valid_unsc, y_valid_unsc,  c= 'dodgerblue',s=1)
ax1.plot([550,900],[550,900], c='black')
ax1.grid()
ax1.set_xlabel('Predicted Temperatures [°C] ', fontsize=15)
ax1.set_ylabel('Measured Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.set(xticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)

In [ ]:
fig, ax1 = plt.subplots(1,  figsize=(5, 5),  sharex=True, sharey=True)
plt.xlim(550, 900)
plt.ylim(550, 900)

ax1.scatter( y_pred_test_unsc_final, y_test_unsc,  c= 'dodgerblue',s=1)
ax1.plot([550,900],[550,900], c='black')
ax1.grid()
ax1.set_xlabel('Predicted Temperatures [°C] ', fontsize=15)
ax1.set_ylabel('Measured Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.set(xticks=[]) 
ax1.yaxis.grid(True)

## looking at single sequences

In [ ]:
## Get predicted and real value for each sequence seperately
seq_ids = test_data.groupby('seq_id')    #group by each sequence+strand_r
seq_id_test_data = list(seq_ids.groups.keys())

df_temp_predicted = pd.DataFrame(y_pred_test_unsc_final, columns=['TempMittelLsInC_predicted','TempMittelFsInC_predicted'])
df_pred = pd.concat([df_temp_predicted, test_data['seq_id']], axis=1)

prediction = {}
real_values = {}
rmse = {}
for seq in seq_id_test_data:
    prediction[seq] = df_pred[['TempMittelLsInC_predicted', 'TempMittelFsInC_predicted']][df_pred['seq_id']==seq].to_numpy()
    real_values[seq] = test_data[['TempMittelLsInC', 'TempMittelFsInC']][test_data['seq_id']
                                                                                   ==seq].to_numpy()
    rmse[seq] = np.sqrt(mean_squared_error( prediction[seq],  real_values[seq]))  

error = []
for seq in seq_id_test_data:
    error.append((rmse[seq], seq))

In [ ]:
sorted(error)

In [ ]:
seq_number = '471949_str_1'
fig1, (ax1) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)

ax1.plot(real_values[seq_number][:,:1], color='black', label='Measured Temperature')
ax1.plot(prediction[seq_number][:,:1], color='blue', label='Predicted Temperature')
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set_ylim(600, 780)
ax1.yaxis.grid(False)
#ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(False)
ax1.legend(prop={'size': 15})
ax1.set(yticks=[])

In [ ]:
seq_number = '473562_str_1'
fig2, (ax2) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)

ax2.plot(real_values[seq_number][:,1:2], color='black', label='Measured Temperature')
ax2.plot(prediction[seq_number][:,1:2], color='blue', label='Predicted Temperature')
ax2.legend(loc='best')
ax2.grid()
ax2.set_xlabel('Minutes', fontsize=15)
ax2.set_ylabel('Temperature [°C]',fontsize=15)
ax2.set_ylim(600, 780)
ax2.yaxis.grid(False)
ax2.xaxis.grid(True)
ax2.legend(prop={'size': 15})
ax1.set(yticks=[])

In [ ]:
seq_number = '475401_str_1'
fig3, (ax1) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)

ax1.plot(real_values[seq_number][:,:1], color='black', label='Measured Temperature')
ax1.plot(prediction[seq_number][:,:1], color='blue', label='Predicted Temperature')
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.xaxis.grid(True)
ax1.set_ylim(600, 780)
ax1.yaxis.grid(False)
ax1.legend(prop={'size': 15})
ax1.set(yticks=[])

In [ ]:
seq_number = '474335_str_2'
fig4, (ax1) = plt.subplots(1, figsize=(6, 5))
sns.lineplot(data=pd.DataFrame(real_values[seq_number][:-1,1:2], columns=['Measured Temperature']),
       palette=['black'],  ax=ax1 )
sns.lineplot(data=pd.DataFrame(prediction[seq_number][:-1,1:2], columns=['Predicted Temperature']),
          palette=['blue'],ax=ax1 )
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.legend(prop={'size': 15})
ax1.set(yticks=[])

In [ ]:
seq_number = '472991_str_2'
fig5, (ax1) = plt.subplots(1, figsize=(6, 5))
sns.lineplot(data=pd.DataFrame(real_values[seq_number][:-1,1:2], columns=['Measured Temperature']),
       palette=['black'],  ax=ax1 )
sns.lineplot(data=pd.DataFrame(prediction[seq_number][:-1,1:2], columns=['Predicted Temperature']),
          palette=['blue'],ax=ax1 )
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.legend(prop={'size': 15})
ax1.set(yticks=[])

In [ ]:
seq_number = '472885_str_2'
fig6, (ax1) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)

ax1.plot(real_values[seq_number][:-1,:1], color='black', label='Measured Temperature')
ax1.plot(prediction[seq_number][:-1,:1], color='blue', label='Predicted Temperature')
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.legend(prop={'size': 15})
ax1.set(yticks=[])